In [1]:
# Import statements for various modules and classes needed for the script.
import os  # Basic operating system functionality
from langchain_core.runnables import (  # Runnable classes for different types of operations
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_core.prompts import ChatPromptTemplate  # Template for chat prompts
from langchain_core.prompts.prompt import PromptTemplate  # Template for prompts
from typing import Tuple, List, Optional  # Type hints for function signatures
from langchain_core.messages import AIMessage, HumanMessage  # Message classes for AI and human interaction
from langchain_core.output_parsers import StrOutputParser  # Parser for string outputs
from langchain_core.runnables import ConfigurableField  # Configurable fields for operations
from yfiles_jupyter_graphs import GraphWidget  # Widget for displaying graphs in Jupyter
from neo4j import GraphDatabase  # Neo4j graph database connection
from langchain_community.vectorstores import Neo4jVector  # Vector store using Neo4j
from langchain_community.graphs import Neo4jGraph  # Graph operations using Neo4j
from langchain.text_splitter import TokenTextSplitter  # Text splitter based on tokens
from langchain_groq import ChatGroq  # Query language for chatting
from langchain_experimental.graph_transformers import LLMGraphTransformer  # Experimental graph transformer
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
import os
from dotenv import load_dotenv

In [2]:
# Load environment variables from the system
NEO4J_URI = os.environ.get("NEO4J_URI")  # Retrieves the Neo4j URI from environment variables
NEO4J_USERNAME = os.environ.get("NEO4J_USERNAME")  # Retrieves the Neo4j username
NEO4J_PASSWORD = os.environ.get("NEO4J_PASSWORD")  # Retrieves the Neo4j password
GROQ_API_KEY = os.environ.get('GROQ_API_KEY')  # Retrieves the Groq API key

# Setting environment variables for Neo4j and GROQ API in the current session
os.environ["NEO4J_URI"] = NEO4J_URI  # Stores the Neo4j URI in the environment variables
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME  # Stores the Neo4j username
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD  # Stores the Neo4j password
os.environ['Groq_api_key'] = GROQ_API_KEY  # Stores the Groq API key in environment variables

In [3]:
# Establish a connection to the Neo4j graph database
graph = Neo4jGraph(
    url=NEO4J_URI,        # Neo4j database URI
    username=NEO4J_USERNAME,  # Username for authentication
    password=NEO4J_PASSWORD,  # Password for authentication
)

C:\Users\SURESH BEEKHANI\AppData\Local\Temp\ipykernel_1996\2843364450.py:2: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(


In [7]:
# Option 1: Using a raw string (if the file is in a 'data' folder within the cwd)
loader = CSVLoader(file_path=r"F:\Multi-AI-Agent-RAG-With-LangGraph\data\legal_research.csv")

# Option 2: Alternatively, using forward slashes
# loader = CSVLoader(file_path="data/legal_research.csv")

docs_list = loader.load()

# Limit to 20 samples
docs_list = docs_list[:20]  

# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=20
)
doc_splits = text_splitter.split_documents(docs_list)

In [8]:
doc_splits

[Document(metadata={'source': 'F:\\Multi-AI-Agent-RAG-With-LangGraph\\data\\legal_research.csv', 'row': 0}, page_content='DocumentID: 1\nCaseName: Case 1\nDocumentType: Regulatory Filing\nDate: 2019-04-02\nSummary: An in-depth analysis of intellectual property is provided in this corporate mergers document.\nRelevanceScore: 90\nKeywords: environmental, regulation, compliance'),
 Document(metadata={'source': 'F:\\Multi-AI-Agent-RAG-With-LangGraph\\data\\legal_research.csv', 'row': 1}, page_content='DocumentID: 2\nCaseName: Case 2\nDocumentType: Brief\nDate: 2018-04-13\nSummary: This filing examines risk management and offers insights into litigation strategy.\nRelevanceScore: 79\nKeywords: employment, discrimination, labor'),
 Document(metadata={'source': 'F:\\Multi-AI-Agent-RAG-With-LangGraph\\data\\legal_research.csv', 'row': 2}, page_content='DocumentID: 3\nCaseName: Case 3\nDocumentType: Legal Analysis\nDate: 2021-01-31\nSummary: This document discusses environmental regulations wit

In [13]:

llm = ChatGroq(temperature=0, model="llama-3.3-70b-versatile")

In [9]:
# Initialize a language model (LLM) using ChatGroq with a specified temperature setting.
# "temperature=0" ensures deterministic responses, reducing randomness.
# The model being used is "llama-3.3-70b-versatile".
llm = ChatGroq(temperature=0, model="llama-3.3-70b-versatile")

# Create an instance of LLMGraphTransformer, which will process documents
# into a graph-based structure using the LLM.
llm_transformer = LLMGraphTransformer(llm=llm)

# Convert the previously split text documents into graph-based representations.
# This transformation helps in structuring knowledge for better retrieval.
graph_documents = llm_transformer.convert_to_graph_documents(doc_splits)

In [10]:
graph_documents[:5]

[GraphDocument(nodes=[Node(id='Case 1', type='Case', properties={}), Node(id='Regulatory Filing', type='Documenttype', properties={}), Node(id='2019-04-02', type='Date', properties={}), Node(id='Intellectual Property', type='Concept', properties={}), Node(id='Corporate Mergers', type='Concept', properties={}), Node(id='Environmental', type='Keyword', properties={}), Node(id='Regulation', type='Keyword', properties={}), Node(id='Compliance', type='Keyword', properties={})], relationships=[Relationship(source=Node(id='Case 1', type='Case', properties={}), target=Node(id='Regulatory Filing', type='Documenttype', properties={}), type='DOCUMENT_TYPE', properties={}), Relationship(source=Node(id='Case 1', type='Case', properties={}), target=Node(id='2019-04-02', type='Date', properties={}), type='DATE', properties={}), Relationship(source=Node(id='Case 1', type='Case', properties={}), target=Node(id='Intellectual Property', type='Concept', properties={}), type='ANALYZES', properties={}), Rel

In [11]:
# Add the graph-based documents to the existing knowledge graph.
graph.add_graph_documents(
    graph_documents,   # The structured graph documents generated from the LLM.
    baseEntityLabel=True,  # Enables labeling of base entities for better organization and retrieval.
    include_source=True  # Retains the original source information for traceability.
)

In [6]:
# directly show the graph resulting from the given Cypher query
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

In [5]:
def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    display(widget)
    return widget

In [14]:
showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

GraphWidget(layout=Layout(height='800px', width='100%'))

In [7]:
google_api_key = os.environ.get("GEMINI_API_KEY")


# Use the API key safely
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=google_api_key)

In [8]:
vector_index = Neo4jVector.from_existing_graph(
    embedding=embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [9]:
graph.query("CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

[]

In [10]:

from langchain_core.pydantic_v1 import BaseModel, Field
# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )

c:\conda_envs\agent\lib\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [11]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

In [14]:
entity_chain = prompt | llm.with_structured_output(Entities)


In [15]:
entity_chain.invoke({"question": "'Regulatory Filing ?"}).names

['Regulatory Filing']

In [16]:
def generate_full_text_query(input: str) -> str:
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

In [17]:
# Fulltext index query
def structured_retriever(question: str) -> str:
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [19]:

print(structured_retriever("provide ,Legal Analysis ?"))

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 13, offset: 104} for query: "CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            }\n            RETURN output LIMIT 50\n            "


Documentid: 3 - HAS_DOCUMENT_TYPE -> Legal Analysis
Documentid: 12 - HAS_DOCUMENT_TYPE -> Legal Analysis
Documentid 9 - CLASSIFIED_AS -> Legal Analysis
Documentid 13 - ANALYSIS_OF -> Legal Analysis
Documentid 18 - ANALYZED_BY -> Legal Analysis


In [20]:
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    return final_data

In [21]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

In [22]:
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [23]:
def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

In [24]:
_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatGroq(temperature=0)
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x : x["question"]),
)

In [25]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""

In [26]:
prompt = ChatPromptTemplate.from_template(template)

In [27]:
chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [34]:
chain.invoke({"question":  "What employment law topics are relevant to Case 4?"})

Search query: What employment law topics are relevant to Case 4?


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 13, offset: 104} for query: "CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            }\n            RETURN output LIMIT 50\n            "
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNoti

'Case 4 is related to employment law, and based on the provided information, the relevant topics include environmental regulations and their implications for employment law.'